# IMPLEMENTING DEEPDREAM

IMPORTING LIBRARIES

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import *

In [13]:
class DeepDreamer(object):
    def __init__(self,
                octave_scale = 1.30, # Determines how much the image size changes between "octaves" (levels of detail/resolution).
                octave_power_factors = None, # Controls the range of octaves, default is [-2, -1, 0, 1, 2]
                layers = None):

        '''
The constructor parameters specify the scale by which we'll increase the size of
an image (octave_scale), as well as the factor that will applied to the scale
(octave_power_factors). layers contains the target layers that will be used
to generate the dreams. Next, let's store the parameters as object members:
        '''
        self.octave_scale = octave_scale
        if octave_power_factors is None:
            self.octave_power_factors = [*range(-2, 3)]
        else:
            self.octave_power_factors = octave_power_factors
        if layers is None:
            self.layers = ["mixed3", "mixed5"]
        else:
            self.layers = layers
        '''
If some of the inputs are None, we use defaults. If not, we use the inputs.
Finally, create the dreamer model by extracting our layers from a pre-trained
InceptionV3 network:
        '''
        self.base_model = InceptionV3(weights = "imagenet",   #  Loads the pre-trained InceptionV3 convolutional neural network, without its classification layer.
                                     include_top = False)
        outputs = [self.base_model.get_layer(name).output
                  for name in self.layers]
        self.dreamer_model = Model(self.base_model.input,
                                  outputs)
        
        

DEFINE A PRIVATE METHOD THAT WILL COMPUTE THE LOSS

In [14]:
def _calculate_loss(self, image):
    image_batch = tf.expand_dims(image, axis = 0) # Adds a batch dimension to the image  into a batch of one image (shape: 1 × height × width × channels).
    activations = self.dreamer_model(image_batch) # urpose: Passes the image through the DeepDream model to get the activations (outputs)

    if len(activations) == 1: # Ensures that activations is always a list, even if there's only one layer.
        activations = [activations]

    losses = []
    for activation in activations:
        loss = tf.math.reduce_mean(activation)
        losses.append(loss)

    total_loss = tf.reduce_sum(losses)
    return total_loss

DEFINE A PRIVATE METHOD THAT WILL PERFORM GRADIENT ASCENT(REMEMBER, WE WANT TO MAGNIFY THE PATTERNS OF THE IMAGE). TO INCREASE PERFORMANCE, WE CAN WRAP THIS FUNCTION IN TF.FUNCTION

In [15]:
@tf.function # This tells TensorFlow to compile the function into a fast, optimized graph for performance.
def _gradient_ascent(self, image, steps, step_size): #  Runs gradient ascent for a set number of steps to "amplify" patterns in the image.
    loss = tf.constant(0.0) # Starts with a loss value of zero. This will be updated each step.

    for _ in range(steps):
        with tf.GradientTape() as tape:
            tape.watch(image)
            loss = self._calculate_loss(image)

        gradeients = tape.gradient(loss, image)
        gradients /= tf.reduce_std(gradients) + le-8

        image = image + gradients * step_size
        image = tf.clip_by_vakue(image, -1, 1)
    return loss, image
        

DEFINE A PRIVATE METHOD THAT WILL CONVERT THE IMAGE TENSOR GENERATED BY THE DREAMER BACK INTO A NUMPY ARRAY

In [16]:
def _deprocess(self, image):
    image = 255 * (image + 1.0) / 2.0 # Neural networks often process images with pixel values normalized to the range [−1,1][−1,1]
    image = tf.cast(image, tf.uint8) # Converts the image data type to uint8, which is the standard for image files
    image = np.array(image) # Converts the TensorFlow tensor to a NumPy array, making it compatible with most image processing libraries and functions.
    return image

DEFINE A PRIVATE METHOD THAT WILL GENERATE A DREAMY IMAGE BY PERFORMING GRADIENT_ASCENT() FOR A SPECOFIC NUMBER OF STEPS

In [19]:
def _dream(self, image, steps, step_size):
    # Converts the input image to the format expected by the neural network (usually scaling pixel values to [−1,1][−1,1] 
    # and converting to a TensorFlow tensor).
    image = preprocess_input(image)
    image = tf.convert_to_tensor(image)

    #  Ensures step_size is a TensorFlow constant, which is required for TensorFlow operations inside the loop.
    step_size = tf.convert_to_tensor(step_size)
    step_size = tf.constant(step_size)

    # Purpose: Tracks how many steps are left and the current step number.
    steps_remaining  = steps
    current_step = 0

    

    #Runs the gradient ascent process in chunks of up to 100 steps at a time (this helps with memory and performance).

    #Calls self._gradient_ascent to update the image, maximizing the activations in the chosen layers.

    #Updates counters after each chunk.

    while steps_remaining > 0:
        if steps_remaining > 100:
            run_steps = tf.constant(100)
        else:
            run_steps = tf.constant(steps_remaining)

        steps_remaining -= run_steps
        current_step += run_steps

        loss, image = self._gradient_ascent(image, run_steps,
                                           step_size)

    # Converts the processed image tensor back to a standard image format
    result = self._deprocess(image)
    return result

DEFINE A PUBLIC METHOD THAT WILL GENERATE DREAMY IMAGES. THE MAIN DIFFERENCE BETWEEN THIS AND _DREAM() (DEFINED IN STEP 6 AND USED INTERNALLY HERE) IS THAT WE WILL USE DIFFERENT IMAGE SIZE (CALLED OCTAVES), AS DETERMINED BY THE ORIGINAL IMAGE SHAPE MULTIPLIED BY A FACTOR, WHICH IS THE PRODUCT OF POWERING SELF.OCTAVE_SCALE TO EACH POWER IN SELF.OCTAVE_POWER_FACTORS:

In [20]:
def dream(self, image, steps = 100, step_size = 0.01):
    image = tf.constant(np.array(image)) # Converts the input image (possibly a PIL or NumPy image) into a TensorFlow tensor
    base_shape = tf.shape(image) [:-1] # Gets the height and width of the image (ignoring the color channels).
    base_shape = tf.cast(base_shape, tf.float32) # Casts the shape to float32 for scaling calculations.
    for factor in self.octave_power_factors:
        new_shape = tf.cast(base_shape*(self.octave_scale ** factor),
                           tf.int32)
        image = tf.image.resize(image,
                               new_shape).numpy()
        image = self._dream(image, steps = steps,
                           step_size = step_size)

        image = tf.image.convert_image_dtype(image / 255.0,
                                            dtype = tf.uint8)
        image = np.array(image)

        return np.array(image)